In [ ]:
# CHUNK 0: VARIABLES INICIALES DE CONFIGURACIÓN
# ======================================================
from pathlib import Path

ROOT = Path.cwd()               # carpeta donde está el .ipynb
DATA_DIR = ROOT                 # los archivos están junto al notebook
OUT_DIR  = ROOT / "outputs"
OUT_DIR.mkdir(exist_ok=True)

# Nombres EXACTOS como aparecen en el panel izquierdo
SECOP_FILE    = "SECOP.csv"
SIIF_FILE     = "SIFF.xlsx"    
DIVIPOLA_FILE = "DIVIPOLA.csv"

# ======================================================
# CHUNK 1: FUNCIONES DE APOYO
# ======================================================
import pandas as pd
import numpy as np

def clean_colnames(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia nombres de columnas: minúsculas, sin espacios, ni tildes.
    """
    df = df.rename(columns=lambda c: (
        str(c)
        .strip()
        .lower()
        .replace(" ", "_")
        .replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")
    ))
    return df

def normalize_money(val):
    """
    Convierte valores de dinero en float seguro.
    """
    if pd.isna(val): 
        return np.nan
    val = str(val).replace(".", "").replace(",", ".")
    val = "".join(ch for ch in val if ch.isdigit() or ch in ".-")
    try:
        return float(val)
    except:
        return np.nan

def normalize_dane(code, length=5):
    """
    Normaliza código DANE a string con ceros a la izquierda.
    """
    if pd.isna(code):
        return None
    code = str(code)
    code = "".join(ch for ch in code if ch.isdigit())
    return code.zfill(length) if code else None

In [ ]:
# CHUNK 2: LECTURA Y LIMPIEZA
def load_secop() -> pd.DataFrame:
    df = pd.read_csv(DATA_DIR / SECOP_FILE, dtype=str, low_memory=False)
    df = clean_colnames(df)
    
    rename = {
        "valor_total": "valor_contrato",
        "municipio_ejecucion": "municipio",
        "departamento_ejecucion": "departamento",
        "codigo_dane_municipio": "cod_dane",
        "modalidad_de_contratacion": "modalidad",
        "nit_entidad": "nit",
        "entidad": "entidad"
    }
    df = df.rename(columns={k:v for k,v in rename.items() if k in df.columns})
    
    if "valor_contrato" in df.columns:
        df["valor_contrato"] = df["valor_contrato"].apply(normalize_money)
    if "cod_dane" in df.columns:
        df["cod_dane"] = df["cod_dane"].apply(normalize_dane)
    
    return df

def load_siif() -> pd.DataFrame:
    df = pd.read_excel(DATA_DIR / SIIF_FILE, dtype=str)
    df = clean_colnames(df)

    possible = {
        "asignado": ["asignado","presupuesto_asignado","apropiacion_inicial"],
        "ejecutado": ["ejecutado","obligaciones","compromisos"],
        "vigencia": ["vigencia","anio","año"],
        "cod_dane": ["cod_dane","codigo_dane","codigo_municipio"]
    }
    for std, candidates in possible.items():
        for c in candidates:
            if c in df.columns:
                df.rename(columns={c: std}, inplace=True)
                break
    
    for c in ["asignado","ejecutado"]:
        if c in df.columns:
            df[c] = df[c].apply(normalize_money)
    if "cod_dane" in df.columns:
        df["cod_dane"] = df["cod_dane"].apply(normalize_dane)
    
    return df

def load_divipola() -> pd.DataFrame:
    df = pd.read_csv(DATA_DIR / DIVIPOLA_FILE, dtype=str)
    df = clean_colnames(df)
    if "cod_dane" in df.columns:
        df["cod_dane"] = df["cod_dane"].apply(normalize_dane)
    return df

In [ ]:
# CHUNK 3: PIPELINE PRINCIPAL (CORREGIDO)

if __name__ == "__main__":
    #Cargar datos ya LIMPIOS desde las funciones de lectura
    secop = load_secop()      # <- DataFrame de SECOP
    siif  = load_siif()       # <- DataFrame de SIIF
    divi  = load_divipola()   # <- DataFrame de DIVIPOLA (claves DANE)

    #Cruce con DIVIPOLA (añade nombres estandarizados de depto/municipio si vienen allí)
    if "cod_dane" in secop.columns:
        secop = secop.merge(divi, on="cod_dane", how="left", suffixes=("", "_div"))
    if "cod_dane" in siif.columns:
        siif  = siif.merge(divi, on="cod_dane", how="left", suffixes=("", "_div"))

    #Indicador de % ejecución (solo si existen las columnas)
    if {"asignado", "ejecutado"}.issubset(siif.columns):
        siif["pct_ejec"] = np.where(siif["asignado"] > 0,
                                    siif["ejecutado"] / siif["asignado"],
                                    np.nan)

    # 4) Exportar resultados
    OUT_DIR.mkdir(exist_ok=True)
    secop.to_csv(OUT_DIR / "secop_limpio.csv", index=False, encoding="utf-8")
    siif.to_csv(OUT_DIR / "siif_limpio.csv",  index=False, encoding="utf-8")

    print("✅ Bases limpias exportadas en carpeta outputs/")


✅ Bases limpias exportadas en carpeta outputs/
